In [19]:
import pandas as pd
import glob
import numpy as np
from tqdm import tqdm
import imblearn
from utils.model_1 import *
from tensorflow import keras
import tensorflow as tf
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
testing_combined=pd.read_parquet('combined_sequence_sampled/training.gzip', engine='auto')
#training_val_combined['labels']=training_val_combined['labels'].astype(int)
testing_combined['measurements']=testing_combined['measurements'].apply(lambda x: np.vstack(x))
X=list(testing_combined['measurements'])
X_padded=tf.keras.utils.pad_sequences(X,dtype=object)

In [20]:
X_balanced_reshaped=tf.convert_to_tensor(X_padded, dtype=tf.float32)

In [72]:
testing_expanded=pd.DataFrame({'measure':list(X_balanced_reshaped.numpy().reshape(X_padded.shape[0]*X_padded.shape[1],-1)),
                 'y_pred':list(np.hstack([np.argmax(y_pred_prob,axis=1)]*20))})

In [58]:
len(list(X_balanced_reshaped.numpy().reshape(X_padded.shape[0]*X_padded.shape[1],-1)))

1341940

In [21]:
model = keras.models.load_model('models2/')

In [44]:
y_pred_prob=model.predict(X_balanced_reshaped)



2097/2097 [==============================] - 19s 9ms/step


In [73]:
testing_expanded

,measure,y_pred
0,"[0.49225122, 0.057482168, -0.5759895, 0.003788...",0
1,"[0.45489722, 0.07778523, -0.5876825, 0.0011984...",0
2,"[0.5086999, 0.08656966, -0.59086084, 0.0018721...",0
3,"[0.5545242, 0.086973965, -0.5889788, 0.0079472...",0
4,"[0.5033685, 0.046890344, -0.5983771, 0.0060456...",0
...,...,...
1341935,"[0.17879802, 0.13687141, -1.6996368, -0.004967...",0
1341936,"[0.18067849, 0.13816367, -1.7034823, -0.004744...",0
1341937,"[0.18048152, 0.13710359, -1.7071319, -0.006291...",0
1341938,"[0.18452206, 0.13680416, -1.7065057, -0.005426...",0


In [22]:
testing_combined=testing_combined[['epoch','measurements','Subject','Session','labels']].rename(columns={'epoch':'time_stamp'})

testing_combined['test_pred']=y_pred

2097/2097 [==============================] - 21s 10ms/step


In [24]:
(0.5-0)/20

0.025

In [23]:
testing_combined

,time_stamp,measurements,Subject,Session,labels,test_pred
0,0.0,"[[0.49225123220935507, 0.0574821663159993, -0....",1,1,0,0
1,0.5,"[[0.5176103850858951, 0.06434617731771121, -0....",1,1,0,0
2,1.0,"[[0.4964102505975528, 0.0902737994839412, -0.5...",1,1,0,0
3,1.5,"[[0.48592396526466025, 0.0707734572733887, -0....",1,1,0,0
4,2.0,"[[0.5142784476363889, 0.06457959309607438, -0....",1,1,0,0
...,...,...,...,...,...,...
67092,1201.0,"[[0.17799350227100835, 0.13431668639868466, -1...",8,1,0,0
67093,1201.5,"[[0.17519039841821377, 0.1363238845497099, -1....",8,1,0,0
67094,1202.0,"[[0.1784703715898555, 0.13786342622431108, -1....",8,1,0,0
67095,1202.5,"[[0.17901440560149806, 0.1329650350252312, -1....",8,1,0,0


In [5]:

#new_combined=testing_combined[['Subject','Session','time_stamp','test_pred']]

2023-04-21 13:59:33.638330: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


2097/2097 [==============================] - 21s 9ms/step


In [9]:
new_combined_s1s1=new_combined[(new_combined.Subject==1)&(new_combined.Session==1)].reset_index(drop=True)[['time_stamp','test_pred']]
new_combined_s1s1['time_stamp']=pd.to_datetime(new_combined_s1s1['time_stamp'],unit='s').round('ms')
new_combined_s1s1=new_combined_s1s1.set_index('time_stamp').asfreq('0.5S')


In [10]:
new_combined_s1s1

,test_pred
time_stamp,
1970-01-01 00:00:00.000,0
1970-01-01 00:00:00.500,0
1970-01-01 00:00:01.000,0
1970-01-01 00:00:01.500,0
1970-01-01 00:00:02.000,0
...,...
1970-01-01 00:15:45.000,0
1970-01-01 00:15:45.500,0
1970-01-01 00:15:46.000,0


In [27]:
original=pd.DataFrame({'time_stamp':list(pd.read_csv('TrainingData/subject_001_01__y_time.csv',header=None)[0]),
                       'labels':list(pd.read_csv('TrainingData/subject_001_01__y.csv',header=None)[0])})
original['time_stamp']=pd.to_datetime(original['time_stamp'],unit='s').round('ms')

In [28]:
original

,time_stamp,labels
0,1970-01-01 00:00:00.020,0
1,1970-01-01 00:00:00.120,0
2,1970-01-01 00:00:00.220,0
3,1970-01-01 00:00:00.320,0
4,1970-01-01 00:00:00.420,0
...,...,...
9468,1970-01-01 00:15:46.820,0
9469,1970-01-01 00:15:46.920,0
9470,1970-01-01 00:15:47.020,0
9471,1970-01-01 00:15:47.120,0


##### sub_9_pred=testing_combined[(testing_combined.Subject==10)&(testing_combined.Session==1)].reset_index(drop=True)
sub_9_pred['time_stamp']=sub_9_pred['time_stamp']+0.02
sub_9=pd.read_csv('TestData/subject_010_01__y_time.csv',header=None).rename(columns={0:'time_stamp'})
#sub_9=pd.concat([sub_9,pd.read_csv('TrainingData/subject_012_01__y.csv',header=None)],axis=1).rename(columns={0:'labels'})
#final=pd.concat([sub_9_pred,sub_9],axis=1)[['test_pred','labels']]
#final['truet']=final['test_pred']==final['labels']
#np.sum(final.truet)/(len(final))
#pd.merge(sub_9,sub_9_pred,how='inner',on='time_stamp')
#sub_9_pred['test_pred'].to_csv('predictions/subject_010_01__y.csv',index=False)


In [ ]:
sub_9_pred

In [ ]:
sub_9

In [ ]:
s=pd.read_csv('/Users/moatasimfarooque/Downloads/subject_012_01__y.csv',header=None)

In [ ]:
s_orf=pd.read_csv('predictions/subject_012_01__y.csv')

In [ ]:
s_orf

In [ ]:
p=s[0]==s_orf['test_pred']

In [ ]:
np.sum(p)/len(p)

In [ ]:
s

In [ ]:
s1=pd.read_csv('predictions/subject_012_01__y.csv')

In [ ]:
len(s1)